In [135]:
import sys
sys.path.append(r'C:\Users\admin\Documents\NiDyN_EEGNET\arl-eegmodels')

In [136]:
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet

In [137]:
from scipy.io import loadmat
from tensorflow.keras.optimizers import Adam

In [138]:
import glob
file_list = glob.glob("Processed_Data\*norm.mat")

In [139]:
import numpy as np
subj = 5

for i in range(0,len(file_list)-1):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    elif i == subj:
        eeg_matrix_test = eeg_matrix
        label_test = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)


In [140]:
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)

In [141]:
num_windows = 10
window_size = 151
step_len = 10

eeg_matrix_train_windowed = np.zeros((eeg_matrix_train.shape[0]*num_windows,1,eeg_matrix_train.shape[2],window_size))
eeg_matrix_test_windowed = np.zeros((eeg_matrix_test.shape[0]*num_windows,1,eeg_matrix_test.shape[2],window_size))
label_train_windowed = np.zeros((label_train.shape[0]*10,label_train.shape[1]))
label_test_windowed = np.zeros((label_test.shape[0]*10,label_test.shape[1]))


for trial in range(eeg_matrix_train.shape[0]):
    for i in range(num_windows):
        eeg_matrix_train_windowed[trial*num_windows+i,:,:,:] = eeg_matrix_train[trial,:,:,step_len*i:step_len*i+151]
        
for trial in range(eeg_matrix_test.shape[0]):
    for i in range(num_windows):
        eeg_matrix_test_windowed[trial*num_windows+i,:,:,:] = eeg_matrix_test[trial,:,:,step_len*i:step_len*i+151]


        
        
label_train_windowed[:,0] = np.repeat(label_train[:,0],10)
label_train_windowed[:,1] = np.repeat(label_train[:,1],10)

label_test_windowed[:,0] = np.repeat(label_test[:,0],10)
label_test_windowed[:,1] = np.repeat(label_test[:,1],10)
        

In [142]:
label_train_windowed.shape

(16380, 2)

In [143]:
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 151)

In [144]:
var_adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])

In [148]:
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train_windowed, y = label_train_windowed, batch_size = 32, epochs = 100, class_weight = class_weights)

Epoch 1/100
16380/16380 [==============================] - 12s 721us/step - loss: 0.9936 - acc: 0.5759
Epoch 2/100
16380/16380 [==============================] - 12s 717us/step - loss: 0.9908 - acc: 0.5740
Epoch 3/100
16380/16380 [==============================] - 12s 717us/step - loss: 0.9832 - acc: 0.5897
Epoch 4/100
16380/16380 [==============================] - 12s 718us/step - loss: 0.9790 - acc: 0.6010
Epoch 5/100
16380/16380 [==============================] - 12s 717us/step - loss: 0.9734 - acc: 0.6064
Epoch 6/100
16380/16380 [==============================] - 12s 718us/step - loss: 0.9710 - acc: 0.6046
Epoch 7/100
16380/16380 [==============================] - 12s 715us/step - loss: 0.9644 - acc: 0.6223
Epoch 8/100
16380/16380 [==============================] - 12s 715us/step - loss: 0.9601 - acc: 0.6239
Epoch 9/100
16380/16380 [==============================] - 12s 715us/step - loss: 0.9593 - acc: 0.6276
Epoch 10/100
16380/16380 [==============================] - 12s 716us/ste

16380/16380 [==============================] - 12s 715us/step - loss: 0.7785 - acc: 0.7516
Epoch 80/100
16380/16380 [==============================] - 12s 715us/step - loss: 0.7773 - acc: 0.7460
Epoch 81/100
16380/16380 [==============================] - 12s 715us/step - loss: 0.7748 - acc: 0.7495
Epoch 82/100
16380/16380 [==============================] - 12s 714us/step - loss: 0.7730 - acc: 0.7462
Epoch 83/100
16380/16380 [==============================] - 12s 716us/step - loss: 0.7694 - acc: 0.7537
Epoch 84/100
16380/16380 [==============================] - 12s 724us/step - loss: 0.7747 - acc: 0.7505
Epoch 85/100
16380/16380 [==============================] - 12s 716us/step - loss: 0.7687 - acc: 0.7554
Epoch 86/100
16380/16380 [==============================] - 12s 715us/step - loss: 0.7712 - acc: 0.7512
Epoch 87/100
16380/16380 [==============================] - 12s 716us/step - loss: 0.7650 - acc: 0.7522
Epoch 88/100
16380/16380 [==============================] - 12s 716us/step - 

In [149]:
a = model.predict(x = eeg_matrix_test_windowed, batch_size = 32)

In [150]:
a[100:200]

array([[0.20462583, 0.7953742 ],
       [0.26069796, 0.7393021 ],
       [0.31974682, 0.68025315],
       [0.4509282 , 0.5490718 ],
       [0.5474253 , 0.45257467],
       [0.58870935, 0.41129065],
       [0.6737723 , 0.32622775],
       [0.71982306, 0.28017688],
       [0.7609662 , 0.23903379],
       [0.76334965, 0.2366504 ],
       [0.5654936 , 0.43450645],
       [0.632364  , 0.36763597],
       [0.7006799 , 0.29932004],
       [0.723346  , 0.276654  ],
       [0.7573081 , 0.24269187],
       [0.7462949 , 0.25370505],
       [0.7253163 , 0.27468365],
       [0.71628   , 0.28372   ],
       [0.74044746, 0.25955257],
       [0.81813294, 0.1818671 ],
       [0.7244938 , 0.27550617],
       [0.7041489 , 0.2958511 ],
       [0.6831366 , 0.31686342],
       [0.6724413 , 0.32755864],
       [0.65093267, 0.3490673 ],
       [0.57436484, 0.42563513],
       [0.5766474 , 0.4233526 ],
       [0.6232875 , 0.37671253],
       [0.66537195, 0.33462802],
       [0.7233288 , 0.2766712 ],
       [0.

In [70]:
model.evaluate(x = eeg_matrix_test, y = label_test)

325/325 [==============================] - 0s 1ms/step


[0.55048073218419, 0.766153846337245]

In [56]:
model.weights.shape

AttributeError: 'list' object has no attribute 'shape'